NOTES:

1. Concatenation of drive data takes a really long time.
2. Although I labeled temps as complete. We should still check whether there is anything inside AERO temps folder that is not in 'verzamelblad.xlsx'

Importing Libraries

In [1]:
#import matplotlib.pyplot as plt
import pandas as pd
#import numpy as np
import glob
import os

Defining Functions

In [2]:
def csv_to_df_concatenator(list_of_paths, axis=None, **kwargs):
    '''
    given a list of paths, it will load all those CSV files and concatenate them 
    based on column names and axis if given 0 or 1.
    kwargs may also be passed to use when reading the files.
    '''
    output_dfs = []
    for csv_file in list_of_paths:
        small_df = pd.read_csv(csv_file, **kwargs)
        output_dfs.append(small_df)
    if axis is not None:
        return pd.concat(output_dfs, axis=axis)
    else:
        return pd.concat(output_dfs)
    
def df_to_feather_compatible(df):
    '''
    checks for mixed type columns and converts them to string columns, 
    since feather can't handle mixed type columns.
    '''
    for col in df.columns:
        if df[col].dtype == "object":
            dtype = pd.api.types.infer_dtype(df[col], skipna=True)
            if dtype in ("mixed", "mixed-integer", "mixed-integer-float"):
                df[col] = df[col].astype(str)
    return df

Handling Exceptions

In [3]:
temps_sheets_exceptions = ['Blad2', 'Blad3']

Reading Datasets

In [4]:
temps_xl = pd.ExcelFile('verzamelblad.xlsx')
SVRM2_drive_csv_files = glob.glob(os.path.join('SVRM2 Drive data', "*.csv"))
SVRM3_drive_csv_files = glob.glob(os.path.join('SVRM3 Drive data', "*.csv"))
SVRM3_ewon_csv_files = glob.glob(os.path.join('SVRM3 Ewon data', "*.csv"))
SVRM4_ewon_csv_files = glob.glob(os.path.join('SVRM4 Ewon data', "*.csv"))

In [5]:
temps_sheets = temps_xl.sheet_names[1:]
for sheet in temps_sheets:
    if sheet in temps_sheets_exceptions:
        small_df = pd.read_excel('verzamelblad.xlsx', sheet)
    else:
        small_df = pd.read_excel('verzamelblad.xlsx', sheet, skiprows=[0,1,2])
    if sheet == temps_sheets[0]:
        temps_df = small_df
    else:
        temps_df = pd.concat((temps_df, small_df))

In [6]:
SVRM2_drive_df = csv_to_df_concatenator(SVRM2_drive_csv_files, axis=0, skiprows=[0], low_memory=False)

In [7]:
SVRM3_drive_df = csv_to_df_concatenator(SVRM3_drive_csv_files, axis=0, skiprows=[0], low_memory=False)

In [8]:
SVRM3_ewon_df = csv_to_df_concatenator(SVRM3_ewon_csv_files, sep=';')

C:\Users\danin\AppData\Local\Temp\ipykernel_20392\3588218242.py:9: DtypeWarning: Columns (3,4,5,6,7,9,14,15,16,20,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  small_df = pd.read_csv(csv_file, **kwargs)


In [9]:
SVRM4_ewon_df = csv_to_df_concatenator(SVRM4_ewon_csv_files, sep=';')

C:\Users\danin\AppData\Local\Temp\ipykernel_20392\3588218242.py:9: DtypeWarning: Columns (3,4,5,6,7,21,24,25,27,28,29,31,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  small_df = pd.read_csv(csv_file, **kwargs)
C:\Users\danin\AppData\Local\Temp\ipykernel_20392\3588218242.py:9: DtypeWarning: Columns (2,3,4,5,6,7,9,10,11,12,13,14,15,16,18,19,20,21,22,23,24,25,27,28,29,33) have mixed types. Specify dtype option on import or set low_memory=False.
  small_df = pd.read_csv(csv_file, **kwargs)
C:\Users\danin\AppData\Local\Temp\ipykernel_20392\3588218242.py:9: DtypeWarning: Columns (2,3,4,5,6,7,9,10,11,12,13,14,15,16,18,19,20,21,22,23,24,25,27,28,29,31,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  small_df = pd.read_csv(csv_file, **kwargs)
C:\Users\danin\AppData\Local\Temp\ipykernel_20392\3588218242.py:9: DtypeWarning: Columns (2,3,4,5,6,7,9,10,11,12,13,14,15,16,18,19,20,21,22,23,24,25,27,28,29,33) have mixed types.

Export DataFrames

In [10]:
temps_df = df_to_feather_compatible(temps_df)
SVRM2_drive_df = df_to_feather_compatible(SVRM2_drive_df)

In [11]:
SVRM3_drive_df = df_to_feather_compatible(SVRM3_drive_df)

In [12]:
SVRM3_ewon_df = df_to_feather_compatible(SVRM3_ewon_df)
SVRM4_ewon_df = df_to_feather_compatible(SVRM4_ewon_df)

In [13]:
temps_df.to_feather('Temps_complete.feather')

In [14]:
SVRM2_drive_df.to_feather('SVRM2_drive_complete.feather')
SVRM3_drive_df.to_feather('SVRM3_drive_complete.feather')

In [15]:
SVRM3_ewon_df.to_feather('SVRM3_ewon_complete.feather')
SVRM4_ewon_df.to_feather('SVRM4_ewon_complete.feather')